In [44]:
from torchvision import datasets,transforms
from sklearn.model_selection import train_test_split
from  torch.utils.data import DataLoader 

In [45]:
trans=transforms.Compose([
    transforms.ToTensor(),#it is like numpy array for pytorch  
    transforms.Normalize((0.5,),(0.5,)) #this normalizes the data between -1 and 1
])
#download the data
train_data=datasets.MNIST(
    root="./data",
    train=True,
    transform=trans,
)
test_data=datasets.MNIST(
    root="./data",
    transform=trans,
    train=False,
)
#print the length of the data
print(len(train_data))
print(len(test_data))


60000
10000


In [46]:
#split the data into train and validation using sklearn
train_data,val_data=train_test_split(train_data,test_size=0.2,random_state=42, stratify=train_data.targets)
print(len(train_data))
print(len(val_data))

48000
12000


In [47]:
#load the data using dataloader
#what does data loder do?it loads the data in batches-it shuffles the data-it makes the data iterable
#what is the batch size?it is the number of samples that will be passed through the network at once
#what is shuffle?it shuffles the data so that the model will not learn the order of the data
train_loader=DataLoader(train_data,batch_size=64,shuffle=True)
val_loader=DataLoader(val_data,batch_size=64,shuffle=True)
test_loader=DataLoader(test_data,batch_size=64,shuffle=True)

In [48]:
print(len(train_loader))
#750 is the number of batches it came from 48000(the length of the train dataset)/64=750 
print(len(val_loader))
#188 is the number of batches it came from 12000(the length of the validation dataset)/64=188
print(len(test_loader))
#157 is the number of batches it came from 10000(the length of the test dataset)/64=157

750
188
157


In [49]:
import torch.nn as n
import torch.nn.functional as f
from torch import optim

In [50]:
# class nn_model(n.Module):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         l1=n.Module.linear(28*28,200)
#         l2=n.Module.linear(200,100)
#         l3=n.Module.linear(100,10)
#     def forward(self,x):
#             x=f.relu(self.l1(x))
#             x=f.relu(self.l2(x))
#             x=self.l3(x)
#             return x
class neural_model(n.Module):
    def __init__(self):
        super().__init__()
        self.layers=n.Sequential(
            n.Linear(28*28,200),
            n.ReLU(),
            n.Linear(200,100),
            n.ReLU(),
            n.Linear(100,10),
            n.LogSoftmax(dim=1)
        )
        self.loss=n.CrossEntropyLoss()
        self.lr=0.01
        self.optimizer=optim.SGD(self.parameters(),lr=self.lr)
    def forward(self,x):
        x=self.layers(x)
        return x
        

In [51]:
model=neural_model()

In [52]:
#train the model
epochs=10
train_losses = []
val_losses = []
train_acc = []
val_acc = []
for epoch in epochs:
    


TypeError: 'int' object is not iterable